# Part 1: Training

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%tensorflow_version 1.x

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print(tf.__version__)

In [ ]:
%cd '/content/drive/My Drive/inception/'
#!git clone https://github.com/tensorflow/models.git

In [ ]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!pip install tf_slim

In [ ]:
%cd /content/drive/My Drive/inception/models/research/
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
import os
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/inception_test/models/research/:/content/drive/My Drive/inception_test/models/research/slim'

In [ ]:
%cd /content/drive/My Drive/inception/models/research
!python setup.py build
!python setup.py install

In [ ]:
%cd /content/drive/My Drive/inception/models/research/slim
!python setup.py build
!python setup.py install

In [ ]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', Left/3600)

In [ ]:
%cd /content/drive/My Drive/inception/models/research/object_detection/builders/
!python model_builder_test.py

In [ ]:
%cd /content/drive/My Drive/inception/models/research/object_detection

In [ ]:
#!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [ ]:
LOG_DIR = 'PLATE_training_dir'
LOG_DIR_2 = 'eval'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR) #changed to eval dir to see accuracies
)

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
!python train.py --logtostderr --train_dir=PLATE_training_dir/ --pipeline_config_path=PLATE_training/faster_rcnn_inception_v2_coco.config

In [ ]:
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path PLATE_training/faster_rcnn_inception_v2_coco.config --trained_checkpoint_prefix PLATE_training_dir/model.ckpt-200000 --output_directory PLATE_inference_graph

# Part 2: Testing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%tensorflow_version 1.x

In [ ]:
# Imports
import cv2
import numpy as np
import os
import sys
# run on CPU
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
from distutils.version import StrictVersion
from collections import defaultdict
import matplotlib.pyplot as plt
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from google.colab.patches import cv2_imshow

In [ ]:
# Model preparation 
PATH_TO_FROZEN_GRAPH = 'frozen_inference_graph.pb'
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'PLATE_labelmap.pbtxt'
NUM_CLASSES = 37

In [ ]:
%cd /content/drive/My Drive/inception/test/
!ls

In [ ]:
# Load a (frozen) Tensorflow model into memory.
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [ ]:
# Detection
def plate_detection(image):
    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:
            # Open image
            image_np = cv2.imread(image)
            #plt.imshow(image_np)
            #plt.show()
            # To get real color we do this:
            image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            # Visualization of the results of a detection.
            (boxes, scores, classes, num_detections) = sess.run(
              [boxes, scores, classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
            vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              line_thickness=1,
              skip_scores=False)
            # Show image with detection
            cv2_imshow(cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB))
            # Save image with detection
            #cv2.imwrite("Predicted_captcha.jpg", cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB))

In [ ]:
plate_detection("AA818.jpg")  

In [ ]:
%cd /content/drive/My Drive/inception/models/research/object_detection
!python eval.py \
    --logtostderr \
    --pipeline_config_path=PLATE_training/faster_rcnn_inception_v2_coco.config \
    --checkpoint_dir=PLATE_training_dir/ \
    --eval_dir=eval/